In [1]:
import pandas as pd
import numpy as np

In [ ]:
subEmbeddings = pd.read_csv('subEmbeddings.tsv', sep="\t" ,header=None)
subEmbeddings.set_index(0, inplace=True)
subEmbeddings

In [2]:
import pickle

with open('../label.pkl', 'rb') as file:
    data = pickle.load(file)

print(data)


{40070: 'licit', 27110: 'licit', 106458: 'licit', 6234: 'suspicious', 26382: 'licit', 17500: 'licit', 86715: 'licit', 5142: 'licit', 10516: 'licit', 1635: 'licit', 11181: 'licit', 4537: 'licit', 90361: 'licit', 38: 'licit', 40071: 'licit', 29686: 'licit', 40072: 'licit', 62071: 'licit', 5319: 'licit', 4090: 'licit', 7409: 'licit', 62072: 'licit', 109677: 'licit', 62073: 'licit', 2730: 'licit', 3938: 'licit', 36423: 'licit', 112712: 'licit', 51090: 'licit', 13400: 'licit', 43371: 'licit', 62074: 'licit', 34329: 'licit', 1177: 'licit', 62075: 'licit', 51254: 'licit', 63046: 'licit', 11182: 'licit', 306: 'licit', 13889: 'licit', 62076: 'licit', 59658: 'licit', 40073: 'licit', 51561: 'licit', 62077: 'licit', 23790: 'licit', 47517: 'licit', 9523: 'licit', 13858: 'licit', 7468: 'licit', 4234: 'licit', 92: 'licit', 11183: 'licit', 54859: 'licit', 40290: 'licit', 4711: 'licit', 119387: 'licit', 62078: 'licit', 29287: 'licit', 1721: 'licit', 62079: 'licit', 236: 'licit', 32865: 'licit', 40452: 

In [3]:
import random

# Calculate 3% of the total number of keys
total_keys = len(data)
sample_size = int(total_keys * 0.03)

# Sample from the keys
sampled_keys = random.sample(list(data.keys()), sample_size)


countIlicit = 0
for k,v in data.items():
    if v == "suspicious":
        countIlicit += 1
        sampled_keys.append(k)
sampled_keys = sorted(sampled_keys)
sampled_keys = set(sampled_keys)
print(countIlicit)
print(len(sampled_keys))

2763
6334


In [49]:
susSamples = 0
nonSusSamples = 0

for k in sampled_keys:
    if data[k] == "suspicious":
        susSamples += 1
    else:
        nonSusSamples += 1
print(susSamples)
print(nonSusSamples)

KeyError: 121811

In [47]:
ccPd = pd.read_csv('dataset/connected_components.csv')
notIncluded = 0
notIncludedKeys = []
included = 0
for k in sampled_keys:
    if k not in ccPd['ccId'].values:
        notIncluded += 1
        notIncludedKeys.append(k)
    else:
        included += 1
print(notIncluded)
print(notIncludedKeys)
print(included)

87
[98489, 33077, 98847, 591, 996, 1182, 2108, 2312, 3116, 3227, 3586, 36673, 4180, 4288, 37520, 5154, 103683, 5392, 6426, 72776, 40919, 41375, 9338, 9515, 75284, 9990, 42956, 10224, 10876, 43661, 10921, 43780, 46259, 14741, 81104, 15885, 16186, 16338, 114657, 82127, 82355, 17052, 17499, 17517, 116028, 17922, 18063, 83733, 51024, 18389, 84021, 18740, 18833, 19456, 118419, 53036, 53252, 20708, 119263, 21218, 54184, 21472, 21958, 87936, 22402, 55246, 22631, 55532, 88871, 56740, 57016, 57127, 90645, 57927, 90914, 58990, 59140, 27784, 60843, 61027, 29210, 62477, 29844, 96508, 31037, 31634, 65375]
6247


In [48]:
susCC = ccPd[ccPd["ccLabel"] == "suspicious"]
licitCC = ccPd[ccPd["ccLabel"] == "licit"]

sample_size = int(len(licitCC) * 0.03)
df_licit_sampled = licitCC.sample(n=sample_size, random_state= 42)

overalSampled = pd.concat([susCC, df_licit_sampled])

sampled_keys = set(overalSampled['ccId'].values)

print(len(sampled_keys))


6334


In [28]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Large DataFrame Processing") \
    .config("spark.driver.memory", "110g")  \
    .getOrCreate()

print("bulding spark session")
# Load only the necessary columns (e.g., the first three)
firstColNodes = spark.read.csv('dataset/background_edges.csv', inferSchema=True, header=True).select('clId1')
print("firstColNodes")
secondColNodes = spark.read.csv('dataset/background_edges.csv', inferSchema=True, header=True).select('clId2')
print("secondColNodes")

occurances = firstColNodes.union(secondColNodes)
print("union")

occurances = occurances.groupBy('clId1').count()
print("groupby")



bulding spark session


firstColNodes


secondColNodes
union
groupby


In [ ]:
for row in occurances.take(5):
    print(row)

In [8]:
filtered_df = occurances.filter(
    occurances.clId1.isin(sampled_keys)
)

In [29]:
occurances.show()

+---------+-----+
|    clId1|count|
+---------+-----+
|742515793|    7|
|692666483|    6|
|743958542|  122|
| 45213064|    3|
|360246048| 5080|
|747313473|    5|
|752365039|    4|
|632092879|   11|
|659765259|    3|
|900711429|   12|
|358649895|    7|
|590623973|   39|
|470660159| 5560|
|702238579|    3|
|722242644|   37|
|746497697|    7|
|748505808|  100|
|611750842|  216|
|744656924|   40|
|753815042|    3|
+---------+-----+
only showing top 20 rows



In [30]:
occurances.count()

49299863

In [31]:
allOccurences = occurances.toPandas()
print("convert is done")
allOccurences.to_csv('allOccurences.csv', index=False)

convert is done


In [50]:
# Load only the necessary columns (e.g., the first three)
subNodes = spark.read.csv('dataset/nodes.csv', inferSchema=True, header=True)
print("read subnodes")

filtered_nodes = subNodes.filter(subNodes.ccId.isin(sampled_keys))
print("filtered nodes")

filtered nodes


In [51]:
filtered_nodes.count()

23541

In [52]:
# Collect the column 'clId' into a list
clId_list = [row['clId'] for row in filtered_nodes.select("clId").collect()]

In [53]:
# Load only the necessary columns (e.g., the first three)
subEdges = spark.read.csv('dataset/edges.csv', inferSchema=True, header=True)
print("read subnodes")

filtered_edges = subEdges.filter(subEdges.clId1.isin(clId_list))
print("filtered edges phase 1")
filtered_edges = filtered_edges.filter(filtered_edges.clId1.isin(clId_list))
print("filtered edges phase 2")

filtered edges phase 1
filtered edges phase 2


In [54]:
filtered_edges.count()

19118

In [55]:
# Load only the necessary columns (e.g., the first three)
connected_components = spark.read.csv('dataset/connected_components.csv', inferSchema=True, header=True)
print("read connected components")

filtered_connected_components = connected_components.filter(connected_components.ccId.isin(sampled_keys))
print("filtered connected components")

filtered connected components


In [56]:
filtered_connected_components.count()

6334

In [57]:
filtered_nodes.toPandas().to_csv('filtered_nodes.csv', index=False)
print("filtered nodes saved")
filtered_edges.toPandas().to_csv('filtered_edges.csv', index=False)
print("filtered edges saved")
filtered_connected_components.toPandas().to_csv('filtered_connected_components.csv', index=False)
print("filtered connected components saved")

filtered nodes saved


filtered edges saved


filtered connected components saved


In [58]:
background_nodes = spark.read.csv('dataset/background_nodes.csv', inferSchema=True, header=True)

print("read background nodes")

filtered_background_nodes = background_nodes.filter(background_nodes.clId.isin(clId_list))

print("filtered background nodes")

read background nodes
filtered background nodes


In [59]:
filtered_background_nodes.count()

23541

In [60]:
filtered_background_nodes.toPandas().to_csv('filtered_background_nodes.csv', index=False)

24/06/08 20:48:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
background_edges = spark.read.csv('dataset/background_edges.csv', inferSchema=True, header=True)

print("read background edges")

filtered_background_edges = background_edges.filter(background_edges.clId1.isin(clId_list))
filtered_background_edges = filtered_background_edges.filter(filtered_background_edges.clId2.isin(clId_list))

print("filtered background edges")


filtered_background_edges.toPandas().to_csv('filtered_background_edges.csv', index=False)

print("filtered background edges saved")

In [25]:
from pyspark.sql.functions import count

# Sample DataFrames
data1 = [(1, 10), (2, 20),(3,70)]
data2 = [(3, 30), (4, 40)]

# Create DataFrames
df1 = spark.createDataFrame(data1, ["clId1", "count"])
df2 = spark.createDataFrame(data2, ["clId2", "count"])

# Perform Union
result_df = df1.union(df2) #df1.union(df2.select(col("clId2").alias("clId1"), "count"))
result_df = result_df.groupBy('clId1').count()
# Show the result
result_df.show()

+-----+-----+
|clId1|count|
+-----+-----+
|    1|    1|
|    2|    1|
|    3|    2|
|    4|    1|
+-----+-----+



In [24]:
from pyspark.sql.functions import col


# Sample DataFrames
data1 = [(1, 10), (2, 20), (1, 30)]
data2 = [(1, 40), (3, 50), (2, 60)]

# Create DataFrames
df1 = spark.createDataFrame(data1, ["clId1", "count"])
df2 = spark.createDataFrame(data2, ["clId2", "count"])

# Perform Union with column names standardized
occurances = df1.select(col("clId1").alias("ID"), "count").union(df2.select(col("clId2").alias("ID"), "count"))


from pyspark.sql.functions import count

# Group by ID and count occurrences
id_counts = occurances.groupBy("ID").count()

# Show the result
id_counts.show()


+---+-----+
| ID|count|
+---+-----+
|  1|    3|
|  2|    2|
|  3|    1|
+---+-----+



In [9]:
filtered_df = filtered_df.withColumnRenamed("clId1", "clId")

In [11]:
filtered_df.write.csv(path='occurrences.csv',
                      mode='overwrite',
                      header=True,
                      sep=',')